In [ ]:
from __future__ import division, unicode_literals, print_function
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np, pandas as pd
import os.path, os, sys, json
import math
from math import sqrt
plt.rcParams.update({'font.size': 18, 'figure.figsize': [10.0, 8.0]})
workdir=%pwd
print (workdir)

## Plot PMF

In [ ]:
%cd $workdir/mc/data

series = "series2"
structurelist = ["closed", "open"]
factlist = [0.05, 0.005]
hlist = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

outprint1=[]
outprint2=[]
plt.plot((0,100),(0,0),'k--',lw=1.0)

for structure in structurelist:
    for fact in factlist:
        for hstr in hlist:
            pfx = str(series)+'-'+str(structure)+'-h'+str(hstr)+'-e'+str(fact)
            %cd -q $pfx

            rdffile="rdf.dat"

            r, g = np.loadtxt(rdffile, unpack=True)         # Load rdf file
            av1=()   
            av2=list(av1)                                      
            for i in np.arange(-20,-1,1):                   # Shift = average of the 20 last values of g 
                av2.append(g[i])
            av3 = sum(av2)/len(av2)  
            g1 = g/av3                                      # Normalized g(r)
            w1 = -np.log(g1)                                # Normalized w(r)

            def b2(x,w):                                    # B2 calculation (here x is the distance)
                inte  = -2*math.pi*(np.exp(-w)- 1)*x**2     # Integration of w(r)
                b2_hc = 2 * math.pi / 3 *(27.8*2)**3        # Hard sphere B2 
                return (np.trapz(inte, x)+b2_hc)/b2_hc      # Normalized B2

            %cd -q ../

        plt.plot( r, w1, lw=2.0, label=pfx)

        pfx2 = str(series)+'-'+str(structure)+'-e'+str(fact)
        outprint1.append(hstr) 
        outprint2.append(b2(r,w1))
        b2file = '../'+"b2-"+pfx2+".dat"  
        np.savetxt(b2file, np.transpose((outprint1,outprint2)))      # Saves B2 in .dat file
    
plt.rcParams.update({'font.size': 18, 'figure.figsize': [10.0, 10.0]})
plt.xlim(20,100)
plt.ylim(-8,4)
plt.xlabel("$r$ ($\mathrm{\AA}$)")
plt.ylabel("$w(r)/kT$")
plt.legend(frameon=False,fontsize=16)

%cd -q $workdir/mc/data

## Plot B2

In [ ]:
%cd $workdir/mc/data

series = "series2"
structurelist = ["closed", "open"]
factlist = [0.05, 0.005]

for structure in structurelist:
    for fact in factlist:
        pfx2 = str(series)+'-'+str(structure)+'-e'+str(fact)
        %cd -q $pfx2

        h,b = np.loadtxt('../'+"b2-"+pfx2+".dat", unpack=True)

        if structure=="closed":  
            mec='r'      
        if structure=="open":
            mec='b'
        if fact==0.05:
            mfc=mec
        if fact==0.005:
            mfc='None'
            
        plt.plot(h,b,ls='None', marker='o',ms=8, mfc=mfc, mec=mec, alpha=1.0, label=str(structure)+", $\epsilon_{LJ}=$"+str(fact)+"$kT$")
        
plt.xlabel("$\epsilon_{HH}=$")
plt.ylabel("$B_2/B_{2HS}$")
plt.legend(frameon=False,fontsize=16)

%cd -q $workdir/mc/data